# Converting LiDAR data into voxels for use in Minetest worlds

## Credit
The following notebook was inspired and adapted from [James Wooton](https://gist.github.com/quantumjim/2d9ea27d0e0428a537b953ac04d3721b).

[Prof. Paul Pickell](https://github.com/pauldpickell) (University of British Columbia) added (October 28, 2021) additional instructions and functionality to change the scale and fill holes in the ground using inverse distance weighting interpolation. Support for additional map layers beyond LiDAR data coming soon.

Licensed under [GNU General Public License v3.0](https://www.gnu.org/licenses/gpl-3.0.en.html).

## Before you get started
- Ensure that you have the packages the following packages installed for python: `pylas`, `numpy`, and `pandas`. You can call `pip install pylas` within your Python IDE, for example.
- [Download Minetest](https://www.minetest.net/downloads/). To install, simply extract the contents somewhere that you have read/write access to on your computer (e.g., your user directory on Windows `C:\Users\YourUsername\minetest\`). Navigate to the `bin` folder, find the `minetest.exe` executable, right-click it and create a shortcut on your desktop or taskbar, then double-click the shortcut to run Minetest.
- [Download the csv2terrain mod](https://github.com/quantumjim/csv2terrain/blob/master/README.md). To install, simply extract the contents into your `mods` folder within your Minetest directory. For example, `C:\Users\YourUsername\minetest\mods\`. Ensure that the folder is renamed `csv2terrain`. Run Minetest and create a new world with any options then press the `Select Mods` button and you should see `csv2terrain` listed there if the installation was done correctly.

In [1]:
import pylas
import numpy as np
import random
import pandas as pd
import math
import datetime
import scipy
from skimage.feature import peak_local_max
from scipy import ndimage

Put this notebook in the same folder as your LiDAR data or otherwise indicate the pathname and filename for your data. This file can be in either `.las` or `.laz` format. It is assumed that the ground has been classified in this LiDAR point cloud.

In [2]:
file = 'MKRF_lidar.las'

## Map centre
Define the centre coordinate of the map. This does not necessarily need to be the centre of the las file, but that is an obvious choice. To find this coordinate, open the las file and calculate the range of the Northings and Eastings, which is usually 1000 or 2000 meters for a typical LiDAR tile. Divide the range by 2 and add that value to the value of the Left extent bound and also to the Bottom extent bound. (Usually LiDAR tiles are square, so you can safely add the same value to the lower bounds of the extent, but you may need to adjust this approach.) 

The Minetest map that you produce is not necessarily going to be the same extent as the LiDAR tile, so you can place the map centre anywhere you want within the las extent. However, you must ensure that this coordinate is at least `size/2` distance from the true edge of the las extent so that you actually build a full map. This coordinate is also used as the starting place for the player.

In [3]:
coord = (533500,5464500)

## Map size
Specify the `size` of the output Minetest map. This is the width and length of the map in blocks. Each block corresponds to 1 meter in the point cloud. Thus, this dimension is what will be used to clip a square extent of `size*size` area from the las file centred at `coord`. Increasing this value does not produce smaller voxels, but instead increases the extent of the Minetest map because the size of blocks in Minetest is fixed. Each incremental value of `size` results in a squared number of new positions to be created and potentially many more blocks, so be careful about creating large maps as they can take long to process and may crash Minetest when loaded. Testing on my Intel i9-10885H yielded about 30 minutes to run a complex 300x300 urban map. You can also try running without the inverse distance weighting interpolation first to see if holes might be a problem in your las. Comment lines 40-56 in the `get_blocks()` function to turn off the interpolation.

In [247]:
size = 300

## Load and preprocess data
`get_points()` applies an `x,y` mask to the point cloud defined by the `size` variable above and then retrieves the `x,y,z` coordinates of all points and rescales the `xy` values to `0 to size` and the `z` values to `0 to max(z)`. Normally, we divide the units of the point cloud by 100 to convert from centimeters to meters, which is approximately what is used by Minetest. You can modify this parameter by choosing any value for `scale`:
- `scale < 1` causes the Minetest world to expand relative to the player (player shrinks)
- `scale = 1` is approximately 1:1 real-world scale
- `scale > 1` causes the Minetest world to shrink relative to the player (player expands)

**Note that if you choose any value other than 1 you will be modifying the scale of the Minetest map (i.e., creating more or fewer voxels relative to the player's perspective).** This parameter does not modify the map `size`. If `scale < 1`, then more voxels are created (point cloud expands *beyond* the map) and any in excess of `size` are dropped and will not be mapped. If `scale > 1`, then fewer voxels are created (point cloud shrinks *within* the map) within the map and some areas of the map `size` may not be used at all. Modifying `scale` should only be used to explore the point cloud data and visualize how different vozel sizes might impact how features might be represented differently.

In [5]:
scale = 1

In [6]:
def get_points(x0,y0,file,size,scale):
    # define scale in terms of units (centimeters)
    scale = int(100*scale)
    # load in the LiDAR point cloud
    cloud = pylas.read(file)
    x, y, z = cloud.x.copy(), cloud.y.copy(), cloud.z.copy()
    # create a mask using the size variable, the LiDAR point cloud will be clipped by this mask
    mask = (x>=x0-size/2) & (x<x0+size/2) & (y>=y0-size/2) & (y<y0+size/2)
    # below for creating dataframe to fill missing values
    cm = pd.DataFrame(cloud.points[mask])
    min_x = min(cm.X)
    min_y = min(cm.Y)
    min_z = min(cm.Z)
    xx = ((cm.X-min_x)/scale).astype(int)
    yy = ((cm.Y-min_y)/scale).astype(int)
    zz = ((cm.Z-min_z)/scale).astype(int)
    c = cm.raw_classification.copy()
    xyzc = pd.concat([xx,yy,zz,pd.Series(c)],axis=1)
    xyzc = xyzc.rename({'raw_classification': 'C'}, axis='columns')
    # remove points with NA for any x, y, z, c value
    xyzc = xyzc.dropna(axis=0)
    # below for fast creation of existing locations with z values using dictionaries (there may be holes to fill)
    height = {}
    classification = {}
    for point in cloud.points[mask]:
        xxx = int((point[0]-min_x)/scale)
        yyy = int((point[1]-min_y)/scale)
        height[xxx,yyy] = int((point[2]-min_z)/scale)
        classification[xxx,yyy] = point[5]
    return height,classification,xyzc

## Block types
Define the block types that you want to associate with the classification values. The typical classification scheme is in the following order:
- 0 = unclassified
- 1 = unassigned
- 2 = ground
- 3 = low vegetation
- 4 = medium vegetation
- 5 = high vegetation
- 6 = buildings
- anything over 6 is noise or reserved for another class

[View all block types (nodes) here.](https://wiki.minetest.net/Games/Minetest_Game/Nodes) 

It is best to use solid blocks and avoid blocks that might not stack well like plants and fungi. When you find a block type you want to use, enter the `Itemstring` for the block that corresponds to the classification value you want to associate it with. The block types below offer nice contrast between vegetation types, buildings and the ground. If you want to experiment with different configurations, find a good starting location (i.e., `coord`) with some variety and set the map `size` to a smaller value (e.g., between 20 and 50) so that you can quickly try out different block types. You can change the top layer of the ground by specifying an `Itemstring` for `top_layer_of_ground_block`. You must specify at least the first 3 classes in order to generate the terrain. You can specify more classes, but any excess classes specified that are not classified in the las file will be ignored and you will see a warning message.

In [7]:
block_type = ['air', # 0 = unclassified
              'steelblock', # 1 = unassigned
              'dirt', # 2 = ground
              'acacia_bush_leaves', # 3 = low vegetation
              'bush_leaves', # 4 = medium vegetation
              'pine_needles', # 5 = high vegetation
              'stonebrick', # 6 = buildings
              'air', # 7 = placeholder
              'air', # 8 = placeholder
              'water_source', # 9 = water
              'air', # 10 = placeholder
              'air', # 11 = placeholder
              'air', # 12 = placeholder
              'air', # 13 = placeholder
              'air'] # 14 = placeholder, add more beyond this if you want

top_layer_of_ground_block = 'dirt_with_grass'
tree_stem_block = 'pine_tree'

## Build and interpolate terrain
`get_blocks()` uses `height`, `classification`, and `zyx` to build the `blocks.csv` file that is then used by the [csv2terrain](https://github.com/quantumjim/csv2terrain/blob/master/README.md) Minetest mod to create the map in Minetest. One important role of this function is to fill any "holes" that might exist in the terrain. For example, ground returns may be incorrectly classified or the ground may be obstructed by a surface feature. In both of these cases, there will be a "hole" in the map because there are no known `z` height values at the given `x,y`. Therefore, inverse distance weighting interpolation is applied to the `x,y` locations that need an interpolated `z` value. Through testing, I have found that `knn=5` nearest neighbours is sufficient, but you can also select any value for `k` below depending on the nature of your input LiDAR point cloud and the classified ground returns.

In [8]:
knn = 5

The function also creates all other classified blocks, including the identification and placement of tree stems and forest canopy. In order to identify the tree stem locations, a local maximum filter is applied to the normalized heights of the canopy (class=5; high vegetation): `canopy elevation - terrain elevation = tree height`. There is a vertical threshold that must be exceeded before a stem is mapped. Since the stem mapping can be sensitive to the type, age, and structure of the forest, the variable `tree_height_percentile` is provided below to control the minimum height of the top of the tree canopy before it may be mapped as a standalone stem. Use `tree_height_percentile = 50` for the median.
**Note: the mapped stems are only for visualization purposes, and may not correspond with the actual location, number or density of true tree stems.**

In [ ]:
tree_height_percentile = 25

In [246]:
def get_blocks(file,coord,size,scale,knn,block_type,top_layer_of_ground_block,tree_stem_block,tree_height_percentile):
    height,classification,xyzc = get_points(coord[0],coord[1],file,size,scale)    
    
    # check that the number of defined block_types matches the classes in the las file and that ground points are classified
    classes = set(classification.values())
    if 2 not in classes:
        print(datetime.datetime.now(),"    Error: No ground points (class=2) found, but expected. Please check that the las file has ground points classified before proceeding. Exiting.")
        return
    elif len(list(classes-set([2])))==0:
        print(datetime.datetime.now(),"    Warning: No other classes found in the las file. If you were expecting more classes other than ground returns, then check your classification and try again.")
    
    # one layer of dirt for the foundation
    blocks = {}
    for x in range(size):
        for y in range(size):
            blocks[x,y,1] = block_type[2]
    
    # start by first building the terrain from the classified ground points
    ground = {key: value for key, value in classification.items() if value == 2}
    groundHeight = {k:height[k] for k in ground.keys()}
    # matrix to hold the value of the highest ground elevation
    terrainHeight = np.zeros((size, size))
    # subset where class=2, we only want to interpolate the ground surface, not heights of other classes
    xyzc_g = xyzc[xyzc['C']==2]
    for x in range(size):
        if x % 10 == 0:
            print(datetime.datetime.now(),"    Processing ground (class=2) line ",x," of ",size)
        for y in range(size):
            # these are the x,y with defined z values
            if (x,y) in groundHeight:
                z = groundHeight[x,y]
                for zz in range(1,z+2):
                    if zz == z+1:
                        # top layer block
                        blocks[x,y,zz] = top_layer_of_ground_block
                        terrainHeight[x,y] = zz
                    else:
                        blocks[x,y,zz] = block_type[2]
            # these are the holes that will be filled with inverse distance weighting interpolation
            else:
                # calculate euclidean distance to all ground points that exist in the point cloud
                ed = np.sqrt((float(x)-xyzc_g['X'].astype(float))**2)+np.sqrt((float(y)-xyzc_g['Y'].astype(float))**2)
                # find the z values of the k nearest neighbours in class 2 only (ground)
                zk = xyzc_g.loc[ed.nsmallest(knn).index,'Z'].astype(float)
                # get the squared euclidean distances (add a small value to ensure no 0 in denominator)
                sd = (ed.nsmallest(knn)**2)+0.0001
                # calculate the inverse distance weighted z value
                z = math.ceil((sum((zk/sd))/sum(1/sd)))
                # fill the hole up to the new z value
                for zz in range(1,z+2):
                    if zz == z+1:
                        # top layer block
                        blocks[x,y,zz] = top_layer_of_ground_block
                        terrainHeight[x,y] = zz
                    else:
                        blocks[x,y,zz] = block_type[2]
    
    # now start building the classified blocks on top of the terrain
    if len(list(classes-set([2])))>0:
        for cl in classes:
            if cl != 2:
                if cl == 5:
                    treeHeight = np.zeros((size, size))
                tempclass = {key: value for key, value in classification.items() if value == cl}
                classHeight = {k:height[k] for k in tempclass.keys()}
                for x in range(size):
                    if x % 10 == 0:
                        print(datetime.datetime.now(),"    Processing class=",cl," line ",x," of ",size)
                    for y in range(size):
                        # these are the x,y with defined z values
                        if (x,y) in classHeight:
                            z = classHeight[x,y]
                            blocks[x,y,z] = block_type[cl]
                            if cl == 5:
                                treeHeight[x,y] = z
                # now we can derive tree canopy shapes (assumed to be class=5; high vegetation)
                if cl == 5:
                    # standardize tree height to terrain height
                    treeHeightNorm = treeHeight-terrainHeight
                    treeHeightNorm[treeHeightNorm<=0] = 0
                    # we map the stems to the max height x,y position and fill blocks down to the ground
                    # here we use the normalized tree heights to counteract the effect of terrain when mapping the stems
                    maxLocalTreeHeight_xy = peak_local_max(treeHeightNorm, min_distance=5,threshold_abs=np.percentile(treeHeightNorm[treeHeightNorm > 0]),n=tree_height_percentile)
                    for (x,y) in classHeight:
                        # get a 3x3 kernel around the target height
                        if x == 0 and y == 0:
                            kern = treeHeight[x:(x+2),y:(y+2)]
                        elif x == size and y == size:
                            kern = treeHeight[(x-1):x,(y-1):y]
                        elif x == 0 and y == size:
                            kern = treeHeight[x:(x+2),(y-1):y]                                
                        elif x == size and y == 0:
                            kern = treeHeight[(x-1):x,y:(y+2)]                                
                        elif x == 0 and y > 0:
                            kern = treeHeight[x:(x+2),(y-1):(y+2)]                                
                        elif x == size and y > 0:
                            kern = treeHeight[(x-1):x,(y-1):(y+2)]                                
                        elif y == 0 and x > 0:
                            kern = treeHeight[(x-1):(x+2),y:(y+2)]
                        elif y == size and x > 0:
                            kern = treeHeight[(x-1):(x+2),(y-1):y]
                        else:
                            kern = treeHeight[(x-1):(x+2),(y-1):(y+2)]
                        kern = kern.astype(int)
                        med = int(np.median(kern[np.nonzero(kern)]))
                        z = int(treeHeight[x,y])
                        # check if the measured height is above or below the local median
                        if z > med:
                            for zz in range(med,z+1):
                                blocks[x,y,zz] = block_type[cl]
                        elif z < med:
                            for zz in range(z,med+1):
                                blocks[x,y,zz] = block_type[cl]
                        elif z == med:
                            # if the height is equivalent to the median then it is likely a single floating voxel
                            # dilate the single floating voxel to make the visual more consistent
                            blocks[x-1,y,z] = block_type[cl]
                            blocks[x+1,y,z] = block_type[cl]
                            blocks[x,y-1,z] = block_type[cl]
                            blocks[x,y+1,z] = block_type[cl]
                            blocks[x,y,z] = block_type[cl]
                    # finally create the blocks for the x,y of the stems
                    for treexy in maxLocalTreeHeight_xy:
                        z = int(treeHeight[treexy[0],treexy[1]])
                        for zz in range(int(terrainHeight[treexy[0],treexy[1]])+1,z-2):
                            blocks[treexy[0],treexy[1],zz] = tree_stem_block
    
    # set the player starting position
    max_height = max(height.values())+2
    x,y = int(size/2),int(size/2)
    if (x,y) in height:
        player = x,height[x,y]+2,y
    else:
        player = x,int(max_height/2),y
        
    # write out the csv file
    print(datetime.datetime.now(),"    Writing out the blocks.csv file now...")
    with open('blocks.csv','w') as file:
        file.write( '0,0,0,min,\n' )
        file.write( str(size)+','+str(max_height)+','+str(size)+','+'max'+',\n' )
        file.write( str(player[0])+','+str(player[1])+','+str(player[2])+','+'player'+',\n' )
        for (x,y,z) in blocks:
            # note that we invert the y-coordinate here with size-y 
            # because this seems to correct the orientation of the world to true cardinal directions
            file.write( str(x)+','+str(z)+','+str(size-y)+','+blocks[x,y,z]+',\n' )

## Run the program

In [248]:
get_blocks(file,coord,size,scale,knn,block_type,top_layer_of_ground_block)

2021-10-31 12:26:07.742801     Processing ground (class=2) line  0  of  300
2021-10-31 12:26:33.321659     Processing ground (class=2) line  10  of  300
2021-10-31 12:26:57.064612     Processing ground (class=2) line  20  of  300
2021-10-31 12:27:17.867077     Processing ground (class=2) line  30  of  300
2021-10-31 12:27:37.161951     Processing ground (class=2) line  40  of  300
2021-10-31 12:27:54.063611     Processing ground (class=2) line  50  of  300
2021-10-31 12:28:09.573492     Processing ground (class=2) line  60  of  300
2021-10-31 12:28:23.963561     Processing ground (class=2) line  70  of  300
2021-10-31 12:28:40.588925     Processing ground (class=2) line  80  of  300
2021-10-31 12:29:00.929166     Processing ground (class=2) line  90  of  300
2021-10-31 12:29:19.553406     Processing ground (class=2) line  100  of  300
2021-10-31 12:29:37.939338     Processing ground (class=2) line  110  of  300
2021-10-31 12:29:55.631929     Processing ground (class=2) line  120  of  3

2021-10-31 12:34:53.816142     Processing class= 7  line  0  of  300
2021-10-31 12:34:53.817140     Processing class= 7  line  10  of  300
2021-10-31 12:34:53.817140     Processing class= 7  line  20  of  300
2021-10-31 12:34:53.817140     Processing class= 7  line  30  of  300
2021-10-31 12:34:53.818138     Processing class= 7  line  40  of  300
2021-10-31 12:34:53.818138     Processing class= 7  line  50  of  300
2021-10-31 12:34:53.818138     Processing class= 7  line  60  of  300
2021-10-31 12:34:53.819134     Processing class= 7  line  70  of  300
2021-10-31 12:34:53.819134     Processing class= 7  line  80  of  300
2021-10-31 12:34:53.819134     Processing class= 7  line  90  of  300
2021-10-31 12:34:53.820132     Processing class= 7  line  100  of  300
2021-10-31 12:34:53.820132     Processing class= 7  line  110  of  300
2021-10-31 12:34:53.820132     Processing class= 7  line  120  of  300
2021-10-31 12:34:53.820132     Processing class= 7  line  130  of  300
2021-10-31 12:34:

## Load the map in Minetest
1. Make sure that you have installed the `csv2terrain` mod in Minetest. See instructions above for download and installation.
2. Once the `blocks.csv` file has been generated, copy the csv file to the `\Minetest\mods\csv2terrain\`. Then, run Minetest and create a new world. Ensure that the `Mapgen` mode is set to `singlenode`, which will generate an empty world with only air blocks (called ["nodes"](https://wiki.minetest.net/Games/Minetest_Game/Nodes) in Minetest). You do not need to provide a seed and make sure `Minetest game` is selected under `Game`. Press the `Create` button to generate the world.
3. Once you are redirected back to the main menu, select your world from the `Select World:` menu. For the best experience, I suggest toggling on `Creative Mode` and toggling off `Enable Damage`. Click the `Select Mods` button and here you should see the `csv2terrain` mod if it was installed correctly. Double-click `csv2terrain` and the text will turn green if it has been enabled properly. Press the `Save` button.
4. Click the `Play Game` button. The player should be falling through the air. Press the `/` key to bring up the console. Type `ltbv` (short for: Let There Be Voxels). Be patient as the Minetest world is being generated from the blocks.csv file. If all goes well, you should land on top of the newly built terrain and you can start navigating in the approximate real-life experience using the arrow keys (or `wasd` keys) and `spacebar` to jump.

## Tips and controls
- Press the `/` key and type `grant singleplayer all` into the console to give the player all game privileges.
- Press the `k` key (default) to enable flying, then press the `spacebar` key to fly up and press `shift` to fly down. Press the `k` key again to disable flying.
- Press the `j` key (default) to enable fast mode, which allows you to move/fly quickly around the world and change perspectives. Press the `j` key again to disable fast mode.
- Press the `/` key and type `time 6000` to change the time of day to 6AM causing the Sun to rise. Increments of 1000 are equivalent to hours in 24-hour time (e.g., 18000 is 6PM).
- Press the `I` key to access the inventory of blocks (nodes) in creative mode. Drag blocks into your inventory space at the bottom. `Right-click` on your mouse (default) to place blocks. `Left-click` on your mouse (default) to mine or remove blocks. Press `B` and `N` keys to move left and right, respectively, in your hotbar inventory of blocks.
- Cardinal directions are true in the Minetest world: North in the Minetest world is true Nouth and the Minetest Sun rises in the true East and sets in the true West.